<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>


1. 네이버 자료

2. 네이버 Place 코드
    * 기준: 서울시 동별, 업종별
    
    
3. 가게정보, 리뷰 정도 겟



# 필요함수 및 라이브러리

In [1]:
import time
import pandas as pd
import numpy as np
import json
from tqdm import tqdm_notebook

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings(action = 'ignore')


## 파일 관련

In [2]:
def get_today():
    import datetime
    now = datetime.datetime.now()
    
    nowDate = now.strftime('%Y-%m-%d')
#     print(nowDate)      # 2015-04-19
 
    nowTime = now.strftime('%H:%M:%S')
#     print(nowTime)      # 12:11:32

    nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
    return nowDate

In [3]:
import os
def check_file(file_path, file_name):
    return os.path.isfile(file_path+file_name)

In [4]:
def save_df(df, path, file_name):
    df.to_csv(path+file_name, index=False)
    print(f'{path+file_name} 저장 완료')

In [5]:
def read_csv_코드정보(code_path , code_file):
    df = pd.read_csv(code_path + code_file )
    return df

In [6]:
'''음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,
   주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, 
   영화공연:CINEMA, 관공서:GOVERNMENT'''

'음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,\n   주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, \n   영화공연:CINEMA, 관공서:GOVERNMENT'

###### 서울시 동 load 관련

In [7]:
import pickle
import collections

def get_seoul_dong():
    with open('data/서울시_구별_동정리.pickle','rb') as fw:
        dong_dict = pickle.load( fw)

    dong_key_list = list(dong_dict.keys())
    dong_key_list.sort()
    # dong_key_list

    for 구이름 in dong_key_list:
        dong_dict[구이름] = sorted(dong_dict[구이름])
    
    return dong_dict

In [8]:
dong_dict = get_seoul_dong()

## 웹 조정 관련

In [9]:
from IPython.display import clear_output

def clear_screen():
    clear_output(wait=True)
    print("Cleared Output")

In [10]:
def is_clickable(wd):
    return wd.is_enabled() and wd.is_displayed()

In [11]:
def scroll_down(wd,SCROLL_PAUSE_SEC = 1):

    # 스크롤 높이 가져옴
    last_height = wd.execute_script("return document.body.scrollHeight")
#     print(last_height)
    while True:
        # 끝까지 스크롤 다운
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = wd.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

## 크롤링 에러 방지 관련

In [12]:
def try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        return wd.find_element_by_class_name(class_name).text
    except NoSuchElementException:
        print(col, end='')
        return np.NaN
    except:
        print('이상한 에러 try_none_text', col)
        return np.NaN
        

In [13]:
def strong_try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        temps = wd.find_elements_by_class_name(class_name)
        for temp in temps:
            if temp.text.startswith('편의'):
                return temp.text.split('\n')[1]
        else:
            return np.NaN
    except NoSuchElementException:
#         print(col, end='')
        return np.NaN
        

In [14]:
def review_dict(s):
    review_dict = {}
#     print(s.split('평균 별점'))
    s = s.split('평균 별점')
    review_dict['평균_별점'] = s[-1].strip()
    s = s[0].split('사진')
    if len(s) >= 2:
        review_dict['사진'] = int(s[-1])
        review_dict['리뷰'] = s[0].split()[-1]
    else:
        review_dict['사진'] = np.NaN
        review_dict['리뷰'] = s[0].split()[-1]
# print(s)
    return review_dict

## get_store_info from naver.place by place_code

In [15]:
def get_review(review_block, name, code_no, show=False):
    reviewer_info = '네이버 업데이트에 의한 삭제'

    r_info_list = review_block.find_element_by_class_name('_3-LAD').find_elements_by_class_name('_1fvo3')

    r_score ,r_date , r_times , proving = np.NaN,np.NaN,np.NaN,np.NaN
    for r_info in r_info_list:
        if '별점' in r_info.text:
            r_score = r_info.text
            continue
        elif '최근' in r_info.text:
            r_date = r_info.text
            continue
        elif '방문' in r_info.text:
            r_times = r_info.text
            continue
        else:
            proving = r_info.text
            continue
    
    r_comment = try_none_text_by_class_name(review_block,'_3vfQ6','r')
    if r_comment:
        pass
    else:
        r_comment = try_none_text_by_class_name(review_block,'WoYOw','r')
        
    owner_reply = try_none_text_by_class_name(review_block,'L-XXo','o')
    
    if show:    
        print(len(r_info_list))
        print('r_score',r_score)
        print('r_date',r_date)
        print('r_times',r_times)
        print('proving',proving)
        print('#### r_comment ####')
        print(r_comment)
        print('####owner_reply###')
        print(owner_reply)
        print('##################')
    return name, code_no, reviewer_info, r_score, r_date, r_times, proving, r_comment, owner_reply

In [16]:
def get_review_none( name, code_no):

    return name, code_no, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN

In [17]:
code_path = 'data/코드/'
구이름 = '강남구'
업종 = 'DINING'
code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
slow=1

code_df = pd.read_csv(code_path+code_file, dtype='str')

code_df['store_code'].to_list()

In [18]:
def naver_review_scrapping(구이름, 업종, slow=1):

    start_time = time.time()

    options = webdriver.ChromeOptions()

    options.add_argument("--start-maximized")
    wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
    wd.implicitly_wait(0.001*slow)

    code_path = 'data/코드/'
    review_path = 'data/리뷰/'


    code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
    review_file = '리뷰정보_' + 구이름 + '_'+ 업종 + '.csv'


    code_df = pd.read_csv(code_path+code_file, dtype='str')

    review_idx = 0
    review_df = pd.DataFrame(columns=("name", "code_no", "reviewer_info", "r_score", "r_date", 'r_times', 'proving', 'r_comments', 'owner_reply'))

    # code_no = code_df.loc[3].values[0]
    store_cnt = 0
    done_idx = 0

    if check_file(review_path, review_file):
        review_df = pd.read_csv(review_path+review_file)
        review_idx = review_df.shape[0]
    done_list = list(set(review_df['code_no'].apply(lambda x: str(x)).values))

    print(f'{구이름}  {업종} 업종 의 리뷰 정보 스크래핑을 시작합니다.')

    print(f'{code_df.shape[0]}개의 가게코드가 있습니다.')

    print(f'{len(done_list)}가게의 리뷰{review_idx}개가 이전에 완료 되었었습니다. 이어서 진행합니다.')
    code_list = code_df['store_code'].to_list()
    
    for code_no_ in tqdm_notebook(code_list):
        code_no = str(code_no_)
        if code_no in done_list:
#             print(code_no, '중복중복!!')
            continue
        store_cnt+=1
        
        place_url = 'https://pcmap.place.naver.com/restaurant/'+str(code_no)+'/review/visitor'
        wd.get(place_url)
        time.sleep(2*slow)
        scroll_down(wd,2)
        cnt_idx = 0
        error_idx = 0 
        wd.implicitly_wait(1*slow)
        
        # 삭제되거나 없어진 페이지 널값 처리
        try:
            if wd.find_element(By.CLASS_NAME, '_1--tV').text.startswith('요청하신'):
                review_df.loc[review_idx] = get_review_none( '삭제된페이지', code_no)
                wd.implicitly_wait(1*slow)
                review_idx += 1
                print('삭제된 페이지')
                continue
        except:
            pass
        

        name = wd.find_element_by_class_name('_3XamX').text
        print('-'*80)
        print(f'{store_cnt} {code_no} {name} 시작', end='||')
    #         print('name',name)
    #         print('code_no',code_no)
        scroll_start_time=time.time()
        try:

            while (time.time() - scroll_start_time) < 100: # 100권장 
    #                 print(time.time() - scroll_start_time)

                wd.find_element_by_class_name('_3iTUo').click()
    #                 print('리뷰 더보기')
                time.sleep(0.5*slow)
            else:
                print(f'{int(time.time() - scroll_start_time)}초 소요','로딩완료', end='||')

        except NoSuchElementException:
            print(f'{int(time.time() - scroll_start_time)}초 소요','로딩완료S', end='||')
        except StaleElementReferenceException :
            print("S_E_Exc")
            pass
        except:
            pass

        펼_idx=0
        for o_bnt in wd.find_elements_by_class_name('dX2wL'):
            try:
                o_bnt.find_element_by_xpath('..').find_element_by_xpath('..').click()
                펼_idx+=1
#                 print('펼', end='')

            except:
                pass
        else:
            print(f'{펼_idx}개 펼치기 완료', end='||')
            
        # 리뷰가 없는 경우
        if wd.find_elements(By.CLASS_NAME, '_1jVSG'):
#             print('리뷰가 있네용~')
            pass
        else:
            print('가게의 리뷰가 없네용~')
            review_df.loc[review_idx] = get_review_none( name, code_no)
            wd.implicitly_wait(1*slow)
            review_idx += 1
            review_df.to_csv(review_path+review_file, index=False)
            continue
            
        review_block_list = wd.find_elements(By.CLASS_NAME, '_1jVSG')[0].find_elements(By.CLASS_NAME, '_3FaRE')
        
        print(f'{len(review_block_list)}개의 리뷰가 있네요')

        
        for review_block in review_block_list:
#             try:
#                 wd.implicitly_wait(0.0001*slow)
#                 review_df.loc[review_idx] = get_review(review_block, name, code_no)
#                 wd.implicitly_wait(1*slow)
#                 review_idx += 1
#                 cnt_idx += 1
#                 done_idx += 1
#                 print('#',end='')
#             except:
#                 print('!',end='')
#                 error_idx += 1
#                 if (error_idx > 30) :
#                     break
#                 pass

            wd.implicitly_wait(0.0001*slow)
            review_df.loc[review_idx] = get_review(review_block, name, code_no)
            wd.implicitly_wait(1*slow)
            review_idx += 1
            cnt_idx += 1
            done_idx += 1
            print('#',end='')            
            
            
            
            
    #             #############################
    #             if cnt_idx > 5:
    #                 break



        ##################### save just in case:
        review_df.to_csv(review_path+review_file, index=False)
        print('\n',구이름, 업종, f'{done_idx}개 중간저장', end='   ')
        print(f'<<{name} {cnt_idx}개 완료.>>')

        if (store_cnt+1)%10 == 0:
            wd.close()
            time.sleep(2*slow)
            wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
            wd.implicitly_wait(1*slow)
            print('reboot chrome')
            
            print('리뷰 수집 데이터 확인해주세요~')
            display(review_df.tail(2))
        


    review_df.to_csv(review_path+review_file, index=False)
    print(f'{구이름} {업종} 의 리뷰 스크래핑을 완료했습니다.')
    print(f'{review_idx} 개 수집완료, {time.time()-start_time}')
    wd.close()

# 오류로 인한 리뷰 정보 null 값 드랍

import pandas as pd
dfs = pd.read_csv('data/리뷰/리뷰정보_강남구_DINING.csv')
dfs.shape
dfs.dropna(axis=0, subset=['r_date', 'reviewer_info'], inplace=True)
dfs.to_csv('data/리뷰/리뷰정보_강남구_DINING.csv',index=False)

구이름 = '강남구'
업종= 'DINING'
naver_review_scrapping(구이름, 업종, slow=1)

# 셋팅

In [19]:
code_path = 'data/코드/'
store_path = 'data/가게/'
review_path = 'data/리뷰/'
# code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
# store_file = '가게정보_' + 구이름 +  '_'+ 업종 + '.csv'
# review_file = '리뷰정보_' + 구이름 +  '_'+ 업종 + '.csv'

In [20]:
detail= True
slow=1

#  코드 수집 및 저장

In [21]:
print(sorted(dong_dict.keys()))

['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


* '강남구' : **세원**


* '강동구', '강북구', '강서구', '관악구' : **준혁**    


* '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구' : **세원**    


* '동작구', '마포구', '서대문구': **준혁**

* '서초구', '성동구', '성북구' : **승순**   


* '송파구', '양천구' : **세원**

     
* '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구':**승순**   


In [22]:
세원_list=['강남구','송파구', '양천구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구' ] 
준혁_list=['강동구', '강북구', '강서구', '관악구', '동작구', '마포구', '서대문구'] 
승순_list=['영등포구', '용산구', '은평구','종로구', '중구', '중랑구', '서초구', '성동구', '성북구'] 

In [ ]:
업종= 'DINING'
idx=0



for i in range(10):
    try:
        for 구이름 in 세원_list:
            code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
            if check_file(code_path, code_file):
                naver_review_scrapping(구이름, 업종, slow=1)
            else:
                print(f'{구이름} {업종}이 없습니다. ')
    except:
        time.sleep(idx**2)
        idx+=1
        print(f'{idx} 실패')
        
    if idx%10 == 0:
        clear_screen()
        
    if idx>=30:
        
        break

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


강남구  DINING 업종 의 리뷰 정보 스크래핑을 시작합니다.
6995개의 가게코드가 있습니다.
6995가게의 리뷰552803개가 이전에 완료 되었었습니다. 이어서 진행합니다.


  0%|          | 0/6995 [00:00<?, ?it/s]

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


강남구 DINING 의 리뷰 스크래핑을 완료했습니다.
552803 개 수집완료, 5.598090648651123
양천구  DINING 업종 의 리뷰 정보 스크래핑을 시작합니다.
3738개의 가게코드가 있습니다.
0가게의 리뷰0개가 이전에 완료 되었었습니다. 이어서 진행합니다.


...

  0%|          | 0/6995 [00:00<?, ?it/s]

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


강남구 DINING 의 리뷰 스크래핑을 완료했습니다.
552803 개 수집완료, 5.5430378913879395
양천구  DINING 업종 의 리뷰 정보 스크래핑을 시작합니다.
3738개의 가게코드가 있습니다.
0가게의 리뷰0개가 이전에 완료 되었었습니다. 이어서 진행합니다.


  0%|          | 0/3738 [00:02<?, ?it/s]

--------------------------------------------------------------------------------
1 1308298219 이키이키 시작||

In [ ]:
df = pd.read_csv('data/리뷰/리뷰정보_강남구_DINING.csv')

In [ ]:
df.shape